In [1]:
"""Machine learning functions"""

import logging
import json
import joblib
import random
import pandas as pd
from pydantic import BaseModel, Field, validator
from sklearn.neighbors import NearestNeighbors



In [2]:

class User(BaseModel):
    """Use this data model to parse the request body JSON."""
    user_input: str = Field(..., example=
        """I am having trouble sleeping, I'm stressed and I don't want to eat. I would like to have an appetite, sleep well, and be able to relax.""")


In [3]:
a_user = User(user_input="I am having trouble sleeping, I'm stressed and I don't want to eat. I would like to have an appetite, sleep well, and be able to relax. I prefer sweet, citrus flavors.}")

In [4]:
from nltk.stem.snowball import SnowballStemmer
sb = SnowballStemmer("english")


In [5]:
nn = joblib.load(r'C:\Users\whats\DS_Unit_3\Build\best-buds\app\jl_knn.joblib')
vectorizer = joblib.load(r'C:\Users\whats\DS_Unit_3\Build\best-buds\app\jl_tfidf.joblib')

MJ = pd.read_csv('https://raw.githubusercontent.com/tt-dspt-200/best-buds/main/data/MJ.csv')


In [10]:
    model_food = vectorizer.transform([sb.stem(a_user.user_input)])
    output = []
    for word in a_user.user_input.split(): 
        output.append(" ".join([sb.stem(word)]))
    stemmed = ""
    for item in output:
        stemmed += item + " "
    model_food = vectorizer.transform({stemmed})

In [11]:
strain_name_list = []
description_list = []
strain_effects_list = []
strain_ailments_list = []

for i in range(5):
    strain_name = (MJ.loc[nn.kneighbors(model_food.todense())[1][0][i]]['Strain']),
    description = (MJ.loc[nn.kneighbors(model_food.todense())[1][0][i]]['Description']),
    strain_effects = (MJ.loc[nn.kneighbors(model_food.todense())[1][0][i]]['Effects_y']),
    strain_ailments = (MJ.loc[nn.kneighbors(model_food.todense())[1][0][i]]['Ailment'])
    strain_name_list.append(strain_name),
    description_list.append(description),
    strain_effects_list.append(strain_effects),
    strain_ailments_list.append(strain_ailments)



In [11]:
strain_df = pd.DataFrame([strain_name_list, description_list, strain_effects_list, strain_ailments_list], index = ['name', 'description', 'effects', 'ailments'],).T
# strain_df = pd.DataFrame([strain_name_list, strain_ailments_list], index = ['name',  'ailments']).T
strain_df

,name,description,effects,ailments
0,"(EAST COAST SOUR DIESEL,)",(East Coast Sour Diesel is a fast-acting attit...,"(Happy, Focused, Energetic, Uplifted, Euphoric...","Muscle Spasms, Stress, Pain, Depression, Infla..."
1,"(NIGHT NURSE,)",(Night Nurse is an indica-dominant strain from...,"(Relaxed, Sleepy, Happy, Tingly, Hungry, Dry M...","Pain, Insomnia, Stress, Muscle Spasms"
2,"(OGRE BERRY,)",(Ogre Berry is an indica-dominant strain culti...,"(Focused, Relaxed, Happy, Sleepy, Euphoric, Dr...","Stress, Lack of Appetite, Insomnia, Pain, Depr..."
3,"(DJANGO,)","(Django means “I awake,” in Romani, and this u...","(Energetic, Uplifted, Happy, Talkative, Euphor...","Stress, Depression, Insomnia, Muscle Spasms"
4,"(BLACK DIAMOND,)",(Black Diamond is a Northern California strain...,"(Relaxed, Happy, Euphoric, Hungry, Uplifted, D...","Stress, Pain, Insomnia, Depression, Inflammation"


In [12]:
strain_df = pd.DataFrame([strain_name_list, description_list, strain_effects_list, strain_ailments_list], index = ['name', 'description', 'effects', 'ailments'],).T
# strain_df = pd.DataFrame([strain_name_list, strain_ailments_list], index = ['name',  'ailments']).T
strain_df


,name,description,effects,ailments
0,"(OGRE BERRY,)",(Ogre Berry is an indica-dominant strain culti...,"(Focused, Relaxed, Happy, Sleepy, Euphoric, Dr...","Stress, Lack of Appetite, Insomnia, Pain, Depr..."
1,"(SUPA DON,)",(Supa Don is a 70% sativa strain bred from Ore...,"(Uplifted, Creative, Energetic, Focused, Happy...","Depression, Insomnia, Stress"
2,"(GREEN POISON,)",(Green Poison is a dangerously flavorful indic...,"(Relaxed, Happy, Hungry, Euphoric, Sleepy, Dry...","Stress, Lack of Appetite, Muscle Spasms, Insom..."
3,"(BLUE ALIEN,)","(Blue Alien is a powerful, sweet-smelling indi...","(Relaxed, Euphoric, Sleepy, Hungry, Happy, Dry...","Pain, Stress, Insomnia, Inflammation"
4,"(SUNSHINE,)",(Sunshine is a sativa-dominant cannabis strain...,"(Uplifted, Energetic, Happy, Euphoric, Creativ...","Stress, Depression, Pain, Nausea"


In [13]:
strain_df.dtypes

name           object
description    object
effects        object
ailments       object
dtype: object

In [14]:
df =strain_df.convert_dtypes()
df.dtypes

name           object
description    object
effects        object
ailments       string
dtype: object

In [16]:
strain_json = strain_df.to_json(orient="records")
strain_json


'[{"name":["OGRE BERRY"],"description":["Ogre Berry is an indica-dominant strain cultivated in the highlands of Northern Mendocino County by HappyDay Farms. Combining Ogre with Island Strawberry Afghani Indica, Ogre Berry regularly tests between 24-28% THC and offers potent relief to aches and pains. Heavy euphoria blankets the mind with sensations of calm relaxation, and while these effects come with great medicinal potential, you may want to set this strain aside when faced with a busy day or cognitive tasks. Sweet citrus flavors meet a fuel-like skunk aroma, creating a sweet and sour fusion to be enjoyed by terpene enthusiasts."],"effects":["Focused, Relaxed, Happy, Sleepy, Euphoric, Dry Mouth, Paranoid"],"ailments":"Stress, Lack of Appetite, Insomnia, Pain, Depression"},{"name":["SUPA DON"],"description":["Supa Don is a 70% sativa strain bred from Oregon Trainwreck (also called \\u201cThe Don\\u201d) and DJ Short Blueberry. Its dark, forest green buds carry a distinct blueberry aro

In [17]:
parsed = json.loads(strain_json)
parsed


[{'name': ['OGRE BERRY'],
  'description': ['Ogre Berry is an indica-dominant strain cultivated in the highlands of Northern Mendocino County by HappyDay Farms. Combining Ogre with Island Strawberry Afghani Indica, Ogre Berry regularly tests between 24-28% THC and offers potent relief to aches and pains. Heavy euphoria blankets the mind with sensations of calm relaxation, and while these effects come with great medicinal potential, you may want to set this strain aside when faced with a busy day or cognitive tasks. Sweet citrus flavors meet a fuel-like skunk aroma, creating a sweet and sour fusion to be enjoyed by terpene enthusiasts.'],
  'effects': ['Focused, Relaxed, Happy, Sleepy, Euphoric, Dry Mouth, Paranoid'],
  'ailments': 'Stress, Lack of Appetite, Insomnia, Pain, Depression'},
 {'name': ['SUPA DON'],
  'description': ['Supa Don is a 70% sativa strain bred from Oregon Trainwreck (also called “The Don”) and DJ Short Blueberry. Its dark, forest green buds carry a distinct bluebe

In [15]:
result = json.dumps(parsed)
result

'[{"name": ["EAST COAST SOUR DIESEL"], "description": ["East Coast Sour Diesel is a fast-acting attitude lifter that patients love for its truly sativa effects. This strain has a sour lemon smell and taste that\\u2019s crisp and uplifting. East Coast Sour Diesel is powerful so new patients should be cautious. Those looking to alleviate anxiety or get some sleep may want to skip this high energy medicine."], "effects": ["Happy, Focused, Energetic, Uplifted, Euphoric, Dry Mouth, Paranoid, Anxious"], "ailments": "Muscle Spasms, Stress, Pain, Depression, Inflammation"}, {"name": ["NIGHT NURSE"], "description": ["Night Nurse is an indica-dominant strain from Reeferman Seeds that combines genetics from BC Hash Plant, Harmony, and Fire OG. It\\u2019s commonly used to fight anxiety, stress, insomnia and chronic pain. Are you having trouble sleeping? This 80/20 indica provides a polite balance of relaxing cerebral and body effects, making this the perfect nighttime sleeping aid."], "effects": [